In [24]:
sequence_leangh = 1000
shellBlock = 10
shellIteration = 4

In [2]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit,transpile, assemble
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import DensityMatrix, partial_trace, Statevector
from qiskit_aer import Aer
import numpy as np
import random

from shell import shell_protocol
from privacyAmlification import generate_hash_family, universal_hash_with_index, secure_hash_using_hmac, get_random_bytes
from depolarization import bipartite_gate, depolarizing_kraus, channel
from BellTest import BellTest

In [3]:
def ChangeBasis(first_basis, second_basis):
    first_eigenvalues, first_eigenvectors = np.linalg.eig(first_basis)
    second_eigenvalues, second_eigenvectors = np.linalg.eig(second_basis)
    U = np.outer(second_eigenvectors[0], first_eigenvectors[0]) + np.outer(second_eigenvectors[1], first_eigenvectors[1])
    return U

In [4]:
def GenerateBellState(eve = False):
    qc = QuantumCircuit(3, 3)
    qc.h(0)
    qc.cx(0, 1)
    if eve:
        qc.cx(1, 2)
        
    return qc

In [5]:
def measure(qc):      
    backend = Aer.get_backend('qasm_simulator')
    transpiled_qc = transpile(qc, backend)
    qobj = assemble(transpiled_qc, shots=1)
    result = backend.run(qc, shots=1).result()
    
    counts = result.get_counts()
    measured_bit = counts.most_frequent()
    return measured_bit

In [6]:
def measure_depolarization(rho):
    outcome = ""
    cdf = [0]
    for i in range(0, rho.shape[0]):
        cdf.append(cdf[i] + rho[i][i])

    rand = np.random.rand()
    
    for i in range(len(cdf)):
        if rand < cdf[i]:
            outcome = format(i-1, '03b')
            break

    return outcome
    
    

In [7]:
def calculate(sequence_leangh, qc, basis, depolarization=False, p=1/2, q=1/2):
    Alice_basis = random.choices([0, 1, 2], [1/3, 1/3, 1/3], k=sequence_leangh)
    Bob_basis = random.choices([1, 2, 3], [1/3, 1/3, 1/3], k=sequence_leangh)
    measured_bits = []
    for i in range(sequence_leangh):

        if depolarization == True:
            rho = np.outer(np.array(Statevector(qc[i])), np.array(Statevector(qc[i])).conj())
            dep_channel = bipartite_gate(depolarizing_kraus(p), depolarizing_kraus(q))
            rho = channel(rho,dep_channel)

            U = ChangeBasis([[1, 0],[0, -1]], basis[Alice_basis[i]])
            U = np.kron(np.kron(np.eye(2), np.eye(2)), U)
            rho = np.matmul(np.matmul(U, rho), U.conj().T)
            
            U = ChangeBasis([[1, 0],[0, -1]], basis[Bob_basis[i]])
            U = np.kron(np.eye(2), np.kron(U, np.eye(2)))
            rho = np.matmul(np.matmul(U, rho), U.conj().T)

            outcome = measure_depolarization(rho)
            measured_bits.append(outcome)
            
        
        elif depolarization == False:
            U = ChangeBasis([[1, 0],[0, -1]], basis[Alice_basis[i]])
            U = Operator(U)
            qc[i].unitary(U, 0)
            U = ChangeBasis([[1, 0],[0, -1]], basis[Bob_basis[i]])
            U = Operator(U)
            qc[i].unitary(U, 1)
            
            qc[i].measure(0,0)
            qc[i].measure(1, 1)
            qc[i].measure(2, 2)
            outcome = measure(qc[i])
            measured_bits.append(outcome)


        

    return Alice_basis, Bob_basis, measured_bits
        

In [8]:
def extract_matching_bits(Alice_basis, Bob_basis, measured_bits):
    weak_key_alice = []
    weak_key_bob = []
    
    for i in range(len(Alice_basis)):
        if Alice_basis[i] == Bob_basis[i]:
            weak_key_alice.append(measured_bits[i][2])
            weak_key_bob.append(measured_bits[i][1])
    
    return weak_key_alice, weak_key_bob

def calculate_difference_percentage(weak_key_alice, weak_key_bob):

    weak_key_alice = [int(bit) for bit in weak_key_alice]
    weak_key_bob = [int(bit) for bit in weak_key_bob]

    differing_bits = sum(1 for i in range(len(weak_key_alice)) if weak_key_alice[i] != weak_key_bob[i])

    difference_percentage = (differing_bits / len(weak_key_alice)) * 100

    return difference_percentage

def compare_and_display_key_difference(weak_key_alice, weak_key_bob):

    difference_percentage = calculate_difference_percentage(weak_key_alice, weak_key_bob)
    print(f"Alice's key: {weak_key_alice}")
    print(f"Bob's key: {weak_key_bob}")
    print(f"Difference between Alice's and Bob's weak keys: {difference_percentage:.2f}%")
    print()

In [17]:
def E91(sequence_leangh, shellBlock, shellIteration, eve = False, depolarization = False, p = 1/2, q = 1/2):
    
    qc = [GenerateBellState(eve) for i in range(sequence_leangh)]
    X = np.array([[0, 1], [1, 0]])
    Z = np.array([[1, 0], [0, -1]])
    theta = 0
    basis = []
    
    basis.append(np.cos(theta) * X + np.sin(theta) * Z)
    basis.append(np.cos(theta + np.pi/4) * X + np.sin(theta + np.pi/4) * Z)
    basis.append(np.cos(theta + np.pi/2) * X + np.sin(theta + np.pi/2) * Z)
    basis.append(np.cos(theta + 3*np.pi/4) * X + np.sin(theta + 3*np.pi/4) * Z)

    Alice_basis, Bob_basis, measured_bits = calculate(sequence_leangh, qc, basis, depolarization, p, q)

    S = BellTest(sequence_leangh, Alice_basis, Bob_basis, measured_bits)
    print("Bell test's experimental value:", S)
    print()
    
    #----------------------------------------------------------#
    
    weak_key_alice, weak_key_bob = extract_matching_bits(Alice_basis, Bob_basis, measured_bits)
    weak_key_alice = ''.join(weak_key_alice)
    weak_key_bob = ''.join(weak_key_bob)
    print("weak keys:")
    compare_and_display_key_difference(weak_key_alice, weak_key_bob)

    #----------------------------------------------------------#
    
    string1 = list(weak_key_alice)
    string2 = list(weak_key_bob)
    s1, s2 = shell_protocol(string1, string2, shellBlock, shellIteration)
    reconcilated_key_Alice = ''.join(s1)
    reconcilated_key_Bob = ''.join(s2) 
    print("reconcilated keys:")
    compare_and_display_key_difference(reconcilated_key_Alice, reconcilated_key_Bob)

    #----------------------------------------------------------#
    
    final_key_length = 32  
    family_size = 256
    key_length = len(weak_key_alice)
    hash_family = generate_hash_family(final_key_length, key_length, family_size)
    
    chosen_hash_index = random.randint(0, family_size - 1)
    print(f"Alice's chosen hash index: {chosen_hash_index}")

    H_alice = hash_family[chosen_hash_index]
    H_bob = hash_family[chosen_hash_index]

    hashed_key_alice = universal_hash_with_index(reconcilated_key_Alice, H_alice)
    hashed_key_bob = universal_hash_with_index(reconcilated_key_Bob, H_bob)
    #print(f"Hashed key for Alice: {hashed_key_alice}")
    #print(f"Hashed key for Bob: {hashed_key_bob}")

    secret_key = get_random_bytes(16)
    final_secure_key_alice = secure_hash_using_hmac(secret_key, hashed_key_alice)
    final_secure_key_bob = secure_hash_using_hmac(secret_key, hashed_key_bob)
    print(f"Final secure key for Alice: {final_secure_key_alice}")
    print(f"Final secure key for Bob: {final_secure_key_bob}")


    

In [18]:
E91(sequence_leangh, shellBlock, shellIteration)

Bell test's experimental value: 2.8812365722988225

weak keys:
Alice's key: 000110110000011001110001100010101111010101100111011111100101100001101110100101010110110110110010001101011111110000011011110010011111001010111011111000110101100010100101001110110001100011011111010101111110010110001101001001110000101001100100101000100
Bob's key: 000110110000011001110001100010101111010101100111011111100101100001101110100101010110110110110010001101011111110000011011110010011111001010111011111000110101100010100101001110110001100011011111010101111110010110001101001001110000101001100100101000100
Difference between Alice's and Bob's weak keys: 0.00%

reconcilated keys:
Alice's key: 011011111000001100011101111001100100101000110110111110100000111001010110111011101110101001001101111100011000100110110010100000100001011101101110010111100001111101010111100110010010100001101101000000011111011000110110101001111010111011011001011011000
Bob's key: 01101111100000110001110111100110010010100011011011111010000011100

In [25]:
E91(sequence_leangh, shellBlock, shellIteration, depolarization = True, p = 0.1, q = 0.1)

Bell test's experimental value: 1.985168974152025

weak keys:
Alice's key: 1100110000110001100101101001011011001000100011011111100011010110010001000101101110100010010001010010001101011101111011000100000010011110100010000000000010111100011110000110101111111100011000011011001010001111
Bob's key: 0100110000010000100100111001011011001000000011111111101010010010111011000011001111000010010001000000001101000101111011010100000010001010100010000000000010111100111110000110111111111100011000011011001010001111
Difference between Alice's and Bob's weak keys: 12.98%

reconcilated keys:
Alice's key: 1111010110110011000010010011010000110011010001000101001011010010101001101111000001100010100000101001011111000111100011000100011101111100101110001001001100111000001100110010111010010110110101001101100110000100
Bob's key: 11110101101100110000100100110100001100110100010001010010110100101010011011110000011000101000001010010111110001111000110001000111011111001011100010010011001110000011001100101110100101101101

In [20]:
E91(sequence_leangh, shellBlock, shellIteration, eve = True)

Bell test's experimental value: 1.6477573624897852

weak keys:
Alice's key: 11101001111101101100011011010110100100110110000001010111110101010001100110001111100101100001111100011011010000010000010101001111011100000001000110110000000010100000110111001100100011111011111100011101111000100101010000000
Bob's key: 11101011011101101100011011010100100100110100000100010111110101000001100101101111000101110001111100011111010000010000010111001111111000000011100110110000100010100000010110001100100011101111111000011101111100100101010000000
Difference between Alice's and Bob's weak keys: 11.31%

reconcilated keys:
Alice's key: 11010011011010110101101000000101100011100000100111010011111010100101111010111010111110011011000010000011000011101010010101110011110110010001100011100111100011000000001001100101000010111101000010110001010001011000101110100
Bob's key: 1101001101101010010110100000010110001110000010011101001111101010010111101011101011111001101100001000001100001110101001010111001111011001000110001110

In [21]:
E91(sequence_leangh, shellBlock, shellIteration, eve = True, depolarization = True, p = 0.1, q = 0.1)

Bell test's experimental value: 1.0909819226139774

weak keys:
Alice's key: 1110111100010010100111010111010011110001000101100001001010001101001100010010101111011011010011101101101110010000111011001010011011001100010101101111111000011000011010010011100000001010110010100110010010101000111001
Bob's key: 1111101100010010100101010111010101110001000101100001001010001101000100011010101111001111110011101111101110011000111111011010111010001110010101110111111010101010011111010011010000001000110111100100000010111000111011
Difference between Alice's and Bob's weak keys: 15.89%

reconcilated keys:
Alice's key: 0101111101000001001110101010111011000100101111001111101001101111010110001111101010101010100000011010000111001100011101001001011100001001101100101100011000111111101001100011000100000011100110011101001000100011010001
Bob's key: 1101111101000001001110101010111011000100101111001011101001101111110110011101101010101010100000011011000111001100011101000001011100001001110101101100011000111111101001100